In [25]:
import itertools
import re
import string
import pandas as pd
import numpy as np
import csv
import math
from autocorrect import spell
from numpy.linalg import inv
from scipy.spatial import KDTree
from nltk import edit_distance
import copy



def checkeditDistance(word1,dict):
    for word2 in dict:
        if(edit_distance(word1,word2) <= 3):
            return True
    return False
    
def wordBreak(s, dict):
    sLen = len(s)
    possible = [False for i in range(sLen + 1)]
    possible[0] = True

    for i in range(sLen):
        for j in range(i + 1):
            if possible[j] and s[j:i + 1] in dict:
                possible[i + 1] = True
                break

    return possible[sLen]
    
    
def computeadaptivekde(place_location,word_loc_dict):
    temp=getxandy(place_location)
    lp1=float(temp[0])
    lp2=float(temp[1])
    rankings=dict()
    for key,value in word_loc_dict.items():
        word=key
        word_location=[]
        for location in value:
            loc=getxandy(location)
            x=float(loc[0])
            y=float(loc[1])
            add=[x,y]
            word_location.append(add)
        if(len(word_location) <= 200 and len(word_location) > 0):
            h=findKnearestneighbour(temp,word_location)
            score=compute(h,value,lp1,lp2)
            rankings[word]=score
    top_words=[]
    temp=sorted(rankings.items(), key=lambda x:x[1])
#     print(temp)
    i=0
    for key,value in temp:
        top_words.append(key)
        i+=1
        if(i==10):
            break
    return top_words
    
    
    
def findKnearestneighbour(placelocation,location_list):
    point=[float(placelocation[0]),float(placelocation[1])]
    if (len(location_list) <= 200):
        tree = KDTree(location_list)
        distances, ndx = tree.query([point], k=5)
        distances[np.isposinf(distances)] = -9999
        p=np.where(distances==-9999.0)
        temp=distances.tolist()
        if len(p[1])==0:
            return temp[0][-1]
        else:
            return temp[0][p[1][0]-1]    

def computekdefixed(place_location,word_loc_dict):
    temp=getxandy(place_location)
    lp1=float(temp[0])
    lp2=float(temp[1])
    rankings=dict()
    for key,value in word_loc_dict.items():
        word=key
        score=compute(10,value,lp1,lp2)
        rankings[word]=score
    top_words=[]
    temp=sorted(rankings.items(), key=lambda x:x[1],reverse=True)
    i=0
    for key,value in temp:
        top_words.append(key)
        i+=1
        if(i==20):
            break
    return top_words
            
#KDE fixed formula for a particular belonging to particular place
def compute(h,locationlist,lp1,lp2):
    C_h=np.matrix([[float(h),float(0)],[float(0),float(h)]])
    C_h_inv=inv(C_h)
    val=[]
    for location in locationlist:
        temp=getxandy(location)
        l1w=float(temp[0])
        l2w=float(temp[1])
        dx=l1w - lp1
        dy=l2w - lp2
        loc_mat=np.matrix([[dx],[dy]])
        loc_mat_transpose=np.transpose(loc_mat)
        inner=(-1/2)*(np.matmul(np.matmul(loc_mat_transpose,C_h_inv),loc_mat))
        guassian=(1/(2*math.pi*h))*(math.exp(inner))
        val.append(guassian)
    return ((1/len(locationlist)) * sum(val))
    
def getxandy(location):
    i=0;
    string=location[location.index('(')+1:location.index(')')]
    return string.split(" ")


def computeaccuracy(top_words):
    counter=0;
    for key,value in top_words.items():
        string_array=key.split();
        dict=[word.lower() for word in string_array]
        for word in value:
            if wordBreak(word.lower(),dict):
                counter+=1
                break
    print((counter/len(top_words))*100)
            


input_data=pd.read_csv('final_data.csv')
del input_data['Unnamed: 0']
input_data.head()
dictionary=dict()

# develop L vector
place_word=dict()
place_loc=dict()
groupbyplaceid=input_data.groupby('PlaceId')
for name,group in groupbyplaceid:
    word_loc=dict()
    for index,row in group.iterrows():
        placelocation=row['Placelocation']
        placename=row['Name']
        placedid=row['PlaceId']
        twitterlocation=row['Twitterlocation']
        tokens=list(row['content'])
        length=len(tokens)
        i=0
        token_list=[]
        while i < length:
            str=""
            if tokens[i] == '\'':
                i+=1
                while(i < length and tokens[i] != '\''):
                    str=str+tokens[i]
                    i+=1
                token_list.append(str)
            i+=1
        #word list for a particular twitter
        #multiple mentions of word in a single tweet count only once i.e. set(token_list)
        string_array=placename.split()
        set_of_tokens=set(token_list)
        set_of_tokens_deepcopy=copy.deepcopy(set_of_tokens)
        for word in set_of_tokens:
            if wordBreak(word.lower(),string_array):
                set_of_tokens_deepcopy.remove(word)
                set_of_tokens_deepcopy.add(''.join(string_array))
            elif checkeditDistance(word,string_array):
                set_of_tokens_deepcopy.remove(word)
                set_of_tokens_deepcopy.add(''.join(string_array))
                
        for word in set_of_tokens_deepcopy:
            if "http" not in word and len(word)>=3:
                #Standardizing words: Sometimes words are not in proper formats. For example: “I looooveee you” should be “I love you”. 
#               Simple rules and regular expressions can help solve these cases.
                word=''.join(''.join(s)[:2] for _, s in itertools.groupby(word))
                word_loc.setdefault(word.lower(), []).append(twitterlocation)
    place_word[placelocation]=word_loc
#     plac
    place_loc[placelocation]=placename


# print(place_word['POINT (1.296346739687204 52.6285572)'])
top_words=dict()

# top_5=computeadaptivekde("POINT (1.296346739687204 52.6285572)",place_word['POINT (1.296346739687204 52.6285572)'])

# print(top_5)

# print(top_5)
# for key,value in place_word.items():
#     top_10=computekdefixed(key,value)
#     top_words[place_loc[key]]=top_10
# computeaccuracy(top_words)

top_words1=dict()
for key,value in place_word.items():
    top_10=computeadaptivekde(key,value)
    top_words1[place_loc[key]]=top_10

# print(top_words1)
computeaccuracy(top_words1)

33.94894079304726
